In [26]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [27]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [28]:
_pwd = ! pwd
_pwd = _pwd[0]
print(_pwd)
file_path = f"file://{_pwd}/Dataset/amazon_reviews.csv"
print(file_path)
# Create an DataFrame from file_path
# people_df = spark.read.csv(file_path, header=True, inferSchema=True)

amzn_df = spark.read.csv(file_path,header=True,inferSchema=True)
amzn_df.show()


amzn_df.describe()




/home/talentum/shared/.ipynb_checkpoints/amzn_prdct_rvw_df
file:///home/talentum/shared/.ipynb_checkpoints/amzn_prdct_rvw_df/Dataset/amazon_reviews.csv
+----------+----------------+-------------+--------------------+-------------------+-----------------+------+
|product_id|product_category|review_rating|         review_text|        review_date|verified_purchase| price|
+----------+----------------+-------------+--------------------+-------------------+-----------------+------+
|      P001|     Electronics|          4.5|Great product and...|2024-01-05 00:00:00|              Yes|599.99|
|      P002|           Books|          4.0|    Informative book|2024-02-12 00:00:00|              Yes|299.99|
|      P003|        Clothing|          3.5|  Good quality shirt|2024-03-09 00:00:00|               No| 49.99|
|      P001|     Electronics|          5.0|Excellent perform...|2024-04-18 00:00:00|              Yes|699.99|
|      P004|           Books|          4.2|Very interesting ...|2024-05-01 00:

DataFrame[summary: string, product_id: string, product_category: string, review_rating: string, review_text: string, verified_purchase: string, price: string]

In [29]:
amzn_df.describe().show()


+-------+----------+----------------+------------------+--------------------+-----------------+----------------+
|summary|product_id|product_category|     review_rating|         review_text|verified_purchase|           price|
+-------+----------+----------------+------------------+--------------------+-----------------+----------------+
|  count|         5|               5|                 5|                   5|                5|               5|
|   mean|      null|            null|              4.24|                null|             null|          399.99|
| stddev|      null|            null|0.5594640292279746|                null|             null|257.390753524675|
|    min|      P001|           Books|               3.5|Excellent perform...|               No|           49.99|
|    max|      P004|     Electronics|               5.0|Very interesting ...|              Yes|          699.99|
+-------+----------+----------------+------------------+--------------------+-----------------+-

In [30]:
from pyspark.sql.functions import (
    avg, count, desc, col
)

In [31]:
#1 product category with the highest avg rating
amzn_df.groupBy("product_category").agg(avg("review_rating").alias("avg_rating")).orderBy(desc("avg_rating")).limit(1).show()


+----------------+----------+
|product_category|avg_rating|
+----------------+----------+
|     Electronics|      4.75|
+----------------+----------+



In [32]:
#2 most frequently reviewed product_id
amzn_df.groupBy("product_id").agg(count("*").alias("review_cnt")).orderBy(desc("review_cnt")).limit(1).show()

+----------+----------+
|product_id|review_cnt|
+----------+----------+
|      P001|         2|
+----------+----------+



In [34]:
#3 avg price for each category
amzn_df.groupBy("product_category").agg(avg("price").alias("avg_price")).show()

+----------------+---------+
|product_category|avg_price|
+----------------+---------+
|     Electronics|   649.99|
|        Clothing|    49.99|
|           Books|   324.99|
+----------------+---------+



In [33]:
#4 verified purchases in each product category
amzn_df.filter(col("verified_purchase") == "Yes").groupBy("product_category").agg(count("*").alias("verified_cnt")).show()

+----------------+------------+
|product_category|verified_cnt|
+----------------+------------+
|     Electronics|           2|
|           Books|           2|
+----------------+------------+

